# Génération de texte avec les réseaux de neurones récurrents LSTM avec Keras

#### Par : BOUTAHIR Mohamed Khalifa [ MASTER IPS ]

---


#### Data : nous allons utiliser un livre de l'enfance comme Data: "Alice's Adventures in Wonderland de Lewis Carroll" . https://gist.github.com/phillipj/4944029


---


#### Le but de projet est de predire une suite où un nouveau scenario pour ce livre .L'utilisation des réseaux LSTM est pour apprendre des séquences de personnages d’Alice au pays des merveilles. Et par la suite le modèle va générer des nouvelles séquences de caractères.
<img src="https://i.ibb.co/MVJLz2B/wonderland.jpg" alt="wonderland" border="0" width="400">

## 1- Importation des bibliothèques et le jeu de données

In [0]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


# 2- Préparation de l’ensemble de données

#### Demande d'accés au drive pour faire l'importation de fichier depuis le



In [0]:
from google.colab import drive
drive.mount('DeepLearning.txt/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at DeepLearning.txt/


#### Importation du fichier texte et le convertir en minuscule

In [0]:
filename = "//content//DeepLearning.txt//My Drive//Colab Notebooks//wonderland generator//wonderland.txt"
text = open(filename).read()
text = text.lower()

#### Detecter les caractères de jeu de donnees et les classer 
 Nous ne pouvons pas modéliser les caractères directement dans le réseau de neurones, nous devons plutôt convertir les caractères en entiers.

In [0]:
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [0]:
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, '*': 7, ',': 8, '-': 9, '.': 10, '0': 11, '3': 12, ':': 13, ';': 14, '?': 15, '[': 16, ']': 17, '_': 18, '`': 19, 'a': 20, 'b': 21, 'c': 22, 'd': 23, 'e': 24, 'f': 25, 'g': 26, 'h': 27, 'i': 28, 'j': 29, 'k': 30, 'l': 31, 'm': 32, 'n': 33, 'o': 34, 'p': 35, 'q': 36, 'r': 37, 's': 38, 't': 39, 'u': 40, 'v': 41, 'w': 42, 'x': 43, 'y': 44, 'z': 45}


#### Calcule le nombre total des caractères  de texte et le nombre des caractères 

In [0]:
n_chars = len(text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  148574
Total Vocab:  46


# 2- Creation de réseau de neurones

Nous allons diviser le texte du livre en sous-séquences d'une longueur fixe de 100 caractères. <br>
Chaque modèle d'apprentissage du réseau est composé de 100 pas de temps d'un caractère (X) suivis d'une sortie de caractère (y).

In [0]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

On va calculer le nombre des échantillons qu'on dans le "X"

In [0]:
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  148474


Maintenant que nous avons préparé nos données d’entraînement, nous devons les transformer pour qu’elles puissent être utilisées avec Keras.

Nous devons d’abord transformer la liste des séquences d’entrée sous la forme [échantillons, pas de temps, fonctions] pour qu'on puisse les entrés pour notre réseau LSTM.

In [0]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

X = X / float(n_vocab)

Chaque valeur y est convertie en un vecteur de longueur 47, plein de zéros, à l’exception d’un 1 dans la colonne correspondant à la lettre (nombre entier représentatif par chaque letre)..

In [0]:
y = np_utils.to_categorical(dataY)

# 3- Définir le modèle LSTM :

Le modéle LSTM utilise 256 unités de mémoire des couches cachées, il utilise aussi Dropout avec une probabilité de 20  

---
Nous avons utilisé aussi une fonction d'activation **Softmax** en couche de sortie 

---

Le model utilise ici l'algorithme d'optimisation **rmsprop** .

In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




---


En raison de la lenteur de l'execution du model et pour l'optimisation, nous allons utiliser un point de contrôle de modèle pour enregistrer tous les poids du réseau à archiver chaque fois qu'une amélioration de la perte est observée à la fin de l'époque. Nous utiliserons le meilleur ensemble de poids (perte minimale) pour instancier notre modèle génératif dans la section suivante.


---



In [0]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

On fait ici une autre transformation des entiers à des caractéres pour qu'on puisse par la suite génerer le texte.

In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))



---


Nous pouvons maintenant adapter notre modèle aux données. Nous utilisons ici un nombre modeste de 20 époques et une grande batch size de 128 patterns .

---



In [0]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
148474/148474 [==============================] - 1563s 11ms/step - loss: 2.9408

Epoch 00001: loss improved from inf to 2.94077, saving model to weights-improvement-01-2.9408.hdf5
Epoch 2/20
148474/148474 [==============================] - 1657s 11ms/step - loss: 2.7063

Epoch 00002: loss improved from 2.94077 to 2.70625, saving model to weights-improvement-02-2.7063.hdf5
Epoch 3/20
148474/148474 [==============================] - 1651s 11ms/step - loss: 2.6000

Epoch 00003: loss improved from 2.70625 to 2.60001, saving model to weights-improvement-03-2.6000.hdf5
Epoch 4/20
148474/148474 [==============================] - 1663s 11ms/step - loss: 2.5304

Epoch 00004: loss improved from 2.60001 to 2.53044, saving model to weights-improvement-04-2.5304.hdf5
Epoch 5/20
148474/148474 [==============================] - 1658s 11ms/step - loss: 2.4708

Epoch 00005: loss improved from 2.53044 to 2.47081, saving model to weights-improvement-05-2.4708.hdf5
Epoch 6/20
148474/148474 [===




---


*   **Nous avons testé le modele dans plusieurs platformes pour trouvé la platforme la plus rapide pour entrainer notre modéle et voila les resultats qu'on a trouvé**


---




<table>
    <tr>
        <td>Platform d'essai</td>
        <td>Temps en (s) pour chaque itération</td>
    </tr>
    <tr>
        <td>GOOGLE COLAB</td>
        <td>850s</td>
    </tr>
    <tr>
        <td>MY LAPTOP</td>
        <td>2000s</td>
    </tr>
    <tr>
        <td>COCALC : https://cocalc.com</td>
        <td>5000s</td>
    </tr>
    <tr>
        <td>AZURE NOTEBOOK</td>
        <td>24394s</td>
    </tr>
</table>

GOOGLE COLAB itération : 
<img src="https://i.ibb.co/xjtWVSJ/Capture3.png" alt="Capture3" border="0">

AZURE itération :
<img src="https://i.ibb.co/S3qNnzY/Capture.png" alt="Capture" border="0">

COLALC itération : 
<img src="https://i.ibb.co/ZKWkPzj/Capture2.png" alt="Capture2" border="0">



---


Nous avons testé deux fonctions d'optimization : 


*   Avec ADAM : le model a reduit le Loss de **3.2198** à **1.9190**
*   Avec RMSPROP : le model a reduit le Loss de **3.3150** à **1.9197**



# 4- Le resultat finale : 

In [0]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" id the king.  `when did you
begin?'

  the hatter looked at the march hare, who had followed him int "
 the was of the wordd     `ii soon hi the mort aro a lan ' said the doypouse.
`and thet so soe toine of the sooe.    the hatter was toe birttenf thet she was notting an in soohe    the hurphon taid to the hocve    `thet io wou doond to the fore   the hatter senlied. `and thet soee of the sorele ' 
  `i c varhin a gitt wite tou,' said the caterpillar.

  `iice toond to the horte     `i aon to the more to taak you oo a                                                                                
Done.
